In [2]:
from sentiment import *

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/vedantdasswain/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vedantdasswain/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/Users/vedantdasswain/Documents/PostDoc/Projects/empathetic_llm/propilot.nosync/venv311/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
test_queries = [
    "Sorry? That's all you've got? A simple \"sorry\" won't fix the mess of a stay I had. What are you going to do about it?",
    "Oh, you \"hear\" me? That's just great. Listening is one thing, but I want action, not just words! What are you going to do about it?.",
    "Well, you better make it quick! And it better be the best room you have, or there will be even more complaints coming your way.",
    "The room that I booked at your hotel was not what was advertised. It was dirty and had a musty smell. I am very disappointed and will not be staying here again.",
    "I appreciate your apology. Could you please let me know what steps can be taken to address this issue?",
    "Thank you for offering to switch me to a clean room. Can you ensure that the new room will be in a better condition than the first one?",
    "That sounds great, thank you. Could you also let me know how long it will take to prepare the new room?",
    "I understand it may take some time, but spending the entire day waiting for a new room is quite inconvenient. Is there any way to expedite this process, or perhaps offer some form of compensation for the inconvenience caused?",
    "That would be appreciated. What form of compensation are you considering?"
]

In [33]:
for query in test_queries:
    print(query)
    print("NLTK:", analyze_sentiment_nltk(query))
    print("TextBlob:", analyze_sentiment_textblob(query))
    print("Transformers:", analyze_sentiment_transformer(test_queries[0]))
    print()

Sorry? That's all you've got? A simple "sorry" won't fix the mess of a stay I had. What are you going to do about it?
NLTK: Slightly Positive
TextBlob: Slightly Negative
Transformers: Very Negative

Oh, you "hear" me? That's just great. Listening is one thing, but I want action, not just words! What are you going to do about it?.
NLTK: Positive
TextBlob: Positive
Transformers: Very Negative

Well, you better make it quick! And it better be the best room you have, or there will be even more complaints coming your way.
NLTK: Very Positive
TextBlob: Positive
Transformers: Very Negative

The room that I booked at your hotel was not what was advertised. It was dirty and had a musty smell. I am very disappointed and will not be staying here again.
NLTK: Very Negative
TextBlob: Very Negative
Transformers: Very Negative

I appreciate your apology. Could you please let me know what steps can be taken to address this issue?
NLTK: Positive
TextBlob: Neutral
Transformers: Very Negative

Thank you 

# Agent Testing

## Updates:
- Customer agent now uses GPT-4o to generate responses
- Historical context for support agents now summarizes the chat history
- Updated prompts for info cue agent to avoid full responses and leverage summary history
- Updated the trouble agent to provide actionable items, avoid general suggestions, leveraging summary history, and send cues as a list

In [30]:
from agents import *

sender_agent = mAgentCustomer()
info_agent = mAgentInfo()
trouble_agent = mAgentTrouble()

In [31]:
def get_history_first():
    chat_history = [AIMessage(content="                AmericanAir I have been trying to get a refund for my canceled flight for over a month now. Every time I call, I am told that it will be processed soon but it has been weeks and I still haven't received anything. This is unacceptable and frustrating. ")]
    return chat_history

def get_history_second():
    chat_history = [AIMessage(content="                AmericanAir I have been trying to get a refund for my canceled flight for over a month now. Every time I call, I am told that it will be processed soon but it has been weeks and I still haven't received anything. This is unacceptable and frustrating. "),
                    HumanMessage(content="sorry for your bad experience, could you please provide your order ID?")]
    return chat_history

prompt = "sorry for your bad experience, could you please provide your order ID?"

In [32]:
result = sender_agent.invoke({"input": prompt, "chat_history": get_history_first(), "civil": 0})
response = result

**Sample Response (POST-UPDATE)** : "Are you serious? You should have all my information lready. It's 12345678. Do your job and fix this mess."

In [33]:
historic_context = sender_agent.get_historical_context_chain().invoke({'question':prompt, "chat_history": get_history_first()})

**Sample Historic Context**: 'Could you please provide your order ID for the refund request regarding your canceled flight?'

### Historic Context

In [34]:
historic_context = info_agent.get_historical_info_context_chain().invoke({'domain': "airlines",'complaint':response, "chat_history": get_history_second()})

In [35]:
historic_context

'My order ID is 123456789. Please process my refund immediately.'

**Sample Historic Context:**
'My order ID is 12345678. Please resolve this issue promptly.'

Output does not always reflect historic context, but rather only the most recent information provided by the customer. The invocation used here already leverages GPT-4o. Therefore, we need to adjust the prompts to reflect the actual context.
Another problem is that the support panes need historical summaries, not just rephrases. 

In [42]:
client_completion = lcai.AzureOpenAI(
    openai_api_key=os.getenv("AZURE_OPENAI_KEY"),
    openai_api_version="2024-05-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment_name="PROPILOT",
    model_name="gpt-4o",
)

def get_historical_context_chain():
    contextualize_q_system_prompt = """
    Your role is to ensure that {message} can be understood without the chat history.\
    
    The chat history contains an online conversation between a customer and a support representative.\
    The {message} is {sender}'s latest response in the chat.\
    
    Summarize the chat history in a way that provides context for the {message}.\
    """
    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{sender}:{message}"),
        ]
    )
    contextualize_q_chain = contextualize_q_prompt | llminfo | StrOutputParser()
    return contextualize_q_chain


In [43]:
historic_context_chat = get_historical_context_chain().invoke({'sender':"representative",'message':prompt, "chat_history": get_history_first()})
print(historic_context_chat)

In the conversation, a customer expressed frustration to a support representative about attempting to obtain a refund for a canceled flight for over a month. Despite being assured by the company on multiple occasions that the refund would be processed soon, the customer has yet to receive it. The representative apologized for the customer's bad experience and requested the order ID to further assist with the issue.


In [44]:
historic_context_info = get_historical_context_chain().invoke({'sender':"client",'message':response, "chat_history": get_history_second()})
print(historic_context_info)

The conversation involves a customer expressing frustration over a delayed refund for a canceled flight with American Airlines. The customer has been attempting to resolve this issue for over a month, repeatedly being assured that the refund would be processed soon, yet has not received it. Upon request from the support representative for further details to assist with the matter, the customer provides their order ID, 123456789, and urges immediate action.


### Info Support Agent

In [48]:
info_agent = mAgentInfo()

response_cw_info = info_agent.invoke({'domain': "airlines",'complaint':response, "chat_history": get_history_second()})
print(response_cw_info)

['Confirm refund policy details.', 'Check the status of the refund process.', 'Apologize and assure resolution.']


**Sample Response:** 
['Apologize for the inconvenience caused.',
 'Confirm receipt of the order ID provided.',
 'Assure investigation and resolution.']
 
The response above is using the old historical context approach and GPT-4. Some of the cues are more about ettiquette and less about the information elements. Sometimes the cues are full responses and not hints.

In [72]:
def extract_cues(chain_output):
    cues_text = chain_output.content
    # Assuming each cue is separated by a newline in the chain_output.
    cues = cues_text.split('\n')
    # Filter out any empty strings or whitespace-only strings
    cues = [cue.strip() for cue in cues if cue.strip()]
    # Return the first 2 - 3 cues
    processed_cues = [re.sub(r'^\d+\.\s*', '', cue) for cue in cues]

    return processed_cues

def agent_coworker_info():
    prompt = """Your role is to help a service representative write a response to a customer they are chatting with online. \

            The representative needs to address the customer's complaint without escalating the issue to a supervisor.\
            The representatives response should have ONE of the following goals:
            1) Inquire more details about the problem. OR \
            2) Request the customer to troubleshoot. OR \
            3) Provide a solution to resolve the customer's need. \
            
            Given the chat history,
            provide 3 hints to help the representative's response.\
            
            Each hint should be a short phrase in a new line.\
            Do NOT number the cues.\
            Do NOT provide the representative with a full response,\
            ONLY provide hints to guide the representative's response.\
            """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{domain}: {message}"),
        ]
    )

    chain = (RunnablePassthrough.assign(
        context=get_historical_context_chain()
    )
             | template
             | llminfo
             )

    chain = chain | extract_cues

    return chain

In [73]:
response_support_info = agent_coworker_info().invoke({'domain': "airlines",'message':response, "chat_history": get_history_second(), "sender": "client"})
print(response_support_info)

["Empathize with the customer's frustration", 'Confirm receipt of the order ID', 'Assure action will be taken promptly']


**Sample Response (POST-UPDATE):**
['Explain the necessity for the order ID to locate the specific booking.', 'Reassure the customer that this will expedite the process.', 'Apologize for the inconvenience and frustration caused.']

### Trouble Support Agent

In [53]:
response_cw_trouble = trouble_agent.invoke({'domain': "airlines",'complaint':response, "chat_history": get_history_second()})

print(response_cw_trouble)

**Sample Response:** 
Step 1: Verify customer's order ID.
Step 2: Review similar cases for resolution.
Step 3: Assess current situation in depth.
Step 4: Explain airline's policy on refunds.
Step 5: Offer alternative solution if refund is not possible.
Step 6: Apologize for inconvenience caused.
Step 7: Ensure resolution is satisfactory to customer.
 
The traditional method does not look at the history correctly. This is why the response does not filter and adjust according to the context. 

In [80]:
def agent_coworker_trouble():
    client = mLangChain()
    prompt = """Your role is to guide a service representative to  resolve the complaint of a customer, to whom they are chatting with online. \

            The representative needs to address the customer's complaint without escalating the issue to a supervisor.\
            
            Review the chat history to understand the steps the representative has taken in response to the complaint. \
            
            List 3-7 items of procedure the representative needs to consider to best service the complaint.\
            
            ONLY list actionable items.\
            AVOID vague or general suggestions.\
            DO NOT list action items that the representative has already taken.\
            Every item should be less than 10 words.\
                            
            Every item should be in a newline.\
                    
                """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{domain}: {message}"),
        ]
    )

    chain = (RunnablePassthrough.assign(
        context=get_historical_context_chain()
    )
            | template
            | llminfo
            )

    chain = chain | extract_cues
    
    return chain

In [81]:
response_cw_trouble = agent_coworker_trouble().invoke({'domain': "airlines",'message':response, "sender": "client", "chat_history": get_history_second()})

print(response_cw_trouble)

["1: Verify the customer's order ID in the system.", '2: Check the status of the refund process.', '3: Apologize for the delay and inconvenience caused.', '4: Provide a clear timeline for the refund completion.', '5: Offer compensation for the inconvenience if applicable.', "6: Confirm the customer's email for sending updates.", '7: Assure follow-up until the issue is resolved.']


### Reframing Agent